In [42]:
import pandas as pd
import re
from nltk.corpus import stopwords
import ast 
from datetime import datetime as dt
import numpy as np

df_trump=pd.read_csv("/home/davide/code/personal/django-ml-runner/django_project/sentiment/tweets_donald_trump.csv",sep=",")#legge csv dei tweet di trump e crea data frame
df_biden=pd.read_csv("/home/davide/code/personal/django-ml-runner/django_project/sentiment/tweets_joe_biden.csv",sep=",")#legge csv dei tweet di biden e crea data frame
df_trump=df_trump.dropna(thresh=2)#rimuove le righe con all'interno almeno 2 NA
df_biden=df_biden.dropna(thresh=2)#rimuove le righe con all'interno almeno 2 NA

def clean_tweet(tweet):
  """"sostituisce con uno spazio vuoto tramite regular esxpression i pattern all'interno delle parentesi"""
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) 

def pre_process_tweets(df):
  """ 
  Preprocess data
  return: dataframe
  """
  df = df[df['text']!="text"]#rimuove nomi colonne nuovi it
  df['text'] = df['text'].apply(clean_tweet)#applico funzione per la pulizia dei tweet
  df.drop(df.columns[7:], axis=1, inplace=True)#cancello colonne inutili. problema causato dal fatto che inizialmente stavo salvando i tweet in modo diverso
  df = df.loc[2:]
  df = df.dropna(thresh=3)#rimuove le righe con all'interno almeno 3 NA
  df = df[df['sentiment'].str.startswith('{')]#scelgo per col 'sentiment' solo le righe che iniziano con "{" perché tweepy restituisce un dizionario
  df['sentiment'] = df['sentiment'].apply(ast.literal_eval)#trasforma una stringa contenente un dizionario, in un dizionario
  df=pd.concat([df.drop(['sentiment'], axis=1), df['sentiment'].apply(pd.Series)], axis=1)#divide il dizionario in delle colonne che hanno come etichetta la chiave del dizionario e all'interno dei campi della colonna vengono inseriti i valori del dizionario
  
  return df 
    


    
df_trump = pre_process_tweets(df_trump)#applico funzione appena creata
df_biden = pre_process_tweets(df_biden)#applico funzione appena creata


/tmp/ipykernel_252169/1621033533.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_tweet)#applico funzione per la pulizia dei tweet
/tmp/ipykernel_252169/1621033533.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.columns[7:], axis=1, inplace=True)#cancello colonne inutili. problema causato dal fatto che inizialmente stavo salvando i tweet in modo diverso
/tmp/ipykernel_252169/1621033533.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [43]:
df_biden

,text,name,location,verified,retweet,date_tweet,sign,polarity
2,RT inflation would be double digits today but ...,K. Andersen,NaN,False,2955,NaN,negative,-0.244444
3,RT Joe Biden has been president for 19 months....,Lynn Walker,NaN,False,735,NaN,positive,0.200000
4,RT BREAKING Bombshell new data reveals that ch...,Joanne,"Grants Pass, OR",False,390,NaN,positive,0.168182
5,RT BREAKING Bombshell new data reveals that ch...,Sleepykat,"Pennsylvania, USA",False,390,NaN,positive,0.168182
6,RT BREAKING Bombshell new data reveals that ch...,Ann Brite,"Bay Minette, AL",False,390,NaN,positive,0.168182
...,...,...,...,...,...,...,...,...
507,RT Joe Biden Gas prices are down $1.30. Realit...,D Matheson,Australia,False,279,2022-09-13 20:42:54+00:00,negative,-0.155556
508,RT _Jordan -Beef -Poultry -Eggs -Milk -Fruit H...,〽️att Leonarddd,NaN,False,807,2022-09-13 20:42:54+00:00,positive,0.500000
509,RT BREAKING Joe Biden will be permitted to tak...,JT🌊💙☮,"North Carolina, USA",False,2517,2022-09-13 20:42:54+00:00,neutral,0.000000
510,RT It's astounding that Joe Biden's only plan ...,Scott Alan,NaN,False,705,2022-09-13 20:42:53+00:00,positive,0.300000


In [44]:
#aggiungo la colonna candidato, mi servirà successivamente quando creerò un unico dataframe dei tweet dei due candidati
df_biden['candidato'] = 'Joe Biden'
df_trump['candidato'] = 'Donald Trump'


In [45]:
import pandas as pd

In [46]:
#percentuali tweet negativi, positivi, neutrali biden
pie_chart_data = pd.DataFrame(df_biden.groupby(['sign','candidato']).size()/df_biden['sign'].count()*100)



In [47]:
pie_chart_data

,,0
sign,candidato,
negative,Joe Biden,17.125984
neutral,Joe Biden,38.976378
positive,Joe Biden,43.897638


In [48]:
# aggiunta colonna con percentuali tweet negativi, positivi, neutrali  trump
a['trump']=df_trump.groupby('sign').size()/df_trump['sign'].count()*100


NameError: name 'a' is not defined

In [ ]:
a.columns=['biden','trump']
a.to_csv('data_for_pie.csv')#salvo i dati in un csv che utilizzerò per i grafici


In [53]:
#concateno i dataframe dei tweet di biden e di trump
df=pd.concat([df_trump, df_biden], ignore_index=True)
df['date_tweet'] = pd.to_datetime(df['date_tweet'],errors='coerce').dt.date
daily_averages = df.groupby(["candidato", "date_tweet"]).aggregate({"polarity":np.mean})
daily_averages=pd.DataFrame(daily_averages).reset_index()
# daily_averages=daily_averages.pivot(index='date_tweet', columns='candidato', values='polarity')
# daily_averages.reset_index(inplace=True)
daily_averages


,candidato,date_tweet,polarity
0,Donald Trump,2022-09-13,0.031397
1,Joe Biden,2022-09-13,0.097552


candidato,Donald Trump,Joe Biden
new_date_column,,
2022-09-13,0.031397,0.097552


In [ ]:
#raggruppo per data e candidato


In [ ]:
#media giornaliera di polarity giornaliera per candidato


In [ ]:
#converto in un dataframe
daily_averages.to_csv('media_polarity.csv', index = True)

In [ ]:
df=pd.read_csv("media_polarity.csv")


In [ ]:
#rimodello il dataset creato, l'indice è la data, le colonne sono i nomi dei candidati e all'interno delle celle troviamo la media giornaliera di polarity


In [ ]:
#visualizzo media polarity
df


,new_date_column,Donald Trump,Joe Biden
0,2020-07-12,0.035279,0.028989
1,2020-07-13,0.005570,0.121669
2,2020-07-14,0.019764,0.076297
3,2020-07-15,0.020163,0.030099
4,2020-07-18,0.027876,0.082130
5,2020-07-19,0.092829,0.082654
6,2020-07-20,-0.028699,0.144445
7,2020-07-21,0.045359,0.072526
8,2020-07-22,0.054454,-0.019329
9,2020-07-23,0.049804,0.073473


In [ ]:
a

,biden,trump
sign,,
negative,22.375935,23.013841
neutral,38.664174,39.815268
positive,38.959891,37.170892
